# Asking user full demo
1. ask product feature
2. ask product condition
3. collect qa -> json
4. 

In [10]:
import sys
sys.path.append('../')
from botcore.setup import get_ai21_model, enable_tracing

MODEL = get_ai21_model(max_tokens = 400)
enable_tracing("ask_demo")

from botcore.chains.qa_feature import build_ask_feature_chain
from botcore.chains.qa_condition import build_ask_condition_chain

from botcore.bot_redis import connect_redis
from botcore.routing.explore_route import FeatureExplorer

product = "phone"
n = 4


AI21 ready
Enable tracing at ask_demo


# RUN

In [11]:

ask_feature = build_ask_feature_chain(MODEL)
ask_condition = build_ask_condition_chain(MODEL)

features = ask_feature({"product": product, "n_top": n})
conditions = ask_condition({"product": product, "n_top": n})


In [12]:
from botcore.utils.json_parser import parse_nested_json
feat = parse_nested_json(features['result'])
feat

{'chain': 'ask_feature',
 'questions': [{'question': 'What is the screen size?',
   'options': ['5.5 inches',
    '6 inches',
    '6.5 inches',
    '7 inches',
    '7.5 inches']},
  {'question': 'What is the RAM size?',
   'options': ['2 GB', '3 GB', '4 GB', '6 GB', '8 GB']},
  {'question': 'What is the storage capacity?',
   'options': ['16 GB', '32 GB', '64 GB', '128 GB', '256 GB']},
  {'question': 'What is the battery capacity?',
   'options': ['2000 mAh', '3000 mAh', '4000 mAh', '5000 mAh', '6000 mAh']}]}

In [13]:
cond = parse_nested_json(conditions['result'])
cond

{'chain': 'ask_condition',
 'questions': [{'question': 'Is the phone fully functional?',
   'options': ['Yes', 'No']},
  {'question': 'Is the phone in good condition?',
   'options': ['Excellent', 'Good', 'Fair', 'Poor']},
  {'question': 'Does the phone have any defects?', 'options': ['Yes', 'No']},
  {'question': 'Is the phone still under warranty?',
   'options': ['Yes', 'No']}]}

# Cache

In [6]:
db = connect_redis()
explorer = FeatureExplorer(MODEL, db)
feat, cond = explorer.ask_user("washing machine", 2)

Features explorer ready


In [7]:
cond

[{'question': 'Has the washing machine ever broken down?',
  'options': ['yes', 'no']},
 {'question': 'Is the washing machine still under warranty?',
  'options': ['yes', 'no']}]

# Load

In [6]:
from botcore.utils.json_parser import parse_nested_json

DATA = parse_nested_json(conditions['result'])
# DATA
ques = [e['question'] for e in DATA]
ans = [e['options'][0] for e in DATA]
bot_memory = QAMemory("question")
bot_memory.load_all(product, ques, ans)


Load done


In [7]:
from botcore.routing.chat_route import ProductChatRouter

router = ProductChatRouter(MODEL,bot_memory)
router.build_router()

Router ready
Build done


In [8]:
ans = router.chain({"input": "Please give a list of pros and cons."})
ans



> Entering new  chain...


/home/h4438/.local/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


pros and cons: {'input': 'Please give a list of pros and cons.', 'question': 'Please give a list of pros and cons.'}

> Entering new  chain...
Prompt after formatting:
You are a secondhand dealer and assessing the user's product. Based on your questions and user answers from the chat history.
 Human: I have this used fridge of mine. Please ask me some questions about it.
AI: Is the fridge making any unusual noises?
Human: yes
AI: Is the fridge cooling properly?
Human: yes
AI: Is the fridge making any unusual noises?
Human: yes
AI: Is the fridge cooling properly?
Human: yes
 Please give your best answer.
 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pros": string  // a js array of the product's pros based on the chat history.
	"cons": string  // a js array of the product's cons based on the chat history.
	"overview": string  // What is your overview on the product.
}
```
 Question: Pl

{'input': 'Please give a list of pros and cons.',
 'question': 'Please give a list of pros and cons.',
 'chat_history': 'Human: I have this used fridge of mine. Please ask me some questions about it.\nAI: Is the fridge making any unusual noises?\nHuman: yes\nAI: Is the fridge cooling properly?\nHuman: yes\nAI: Is the fridge making any unusual noises?\nHuman: yes\nAI: Is the fridge cooling properly?\nHuman: yes',
 'text': '\n```\n{\n "pros": [\n  "large capacity",\n  "energy efficient",\n  "quiet operation"\n ],\n "cons": [\n  "noisy"\n ],\n "overview": "The fridge is large capacity, energy efficient and quiet, but it\'s noisy."\n}\n\n```'}